# iForest

In [1]:
import warnings
warnings.filterwarnings('ignore')

## General libraries

In [2]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from os.path import join
import json
import datetime

import shap
from shap_selection import feature_selection

from sklearn.preprocessing import OneHotEncoder
import time

### Load enviroment variables

In [3]:
from dotenv import load_dotenv
load_dotenv('../.env')

code_root = os.environ['CODE_ROOT']
cfg_path = os.environ['CFG_PATH']
data_root = os.environ['DATA_ROOT']

sys.path.insert(0, code_root)

### Specific libraries

In [4]:
from src.load.functions import get_fs_dataset, fs_datasets_hyperparams
from src.plots.functions import plot_3d_surface, plot_2d_surface, boxplot_stability, lineplot_stability
from src.utils.functions import add_custom_repeating_sequence, add_sequence_to_dataframe, prepare_subsets
from src.optimization.functions import opt_value

# if we want to create an iteractive plot or not
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

## Parameters

In [5]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch


import os
import pandas as pd
import numpy as np

def process_data(dataset_id, df):
    """
    Processes SHAP data files by reading them, filtering, and calculating additional metrics including the interquartile range (IQR).
    
    Parameters:
    - dataset_id: The unique identifier for the dataset.
    - data_root: The root directory where the data files are located.
    
    Returns:
    - df: The processed pandas DataFrame containing selected variables and calculated metrics.
    """

    # Filter dataframe
    df = df[(df.n_estimators >= 25) & (df.n_estimators <= 300)]
    
    # Calculate percentiles, stability index, and IQR
    df['shap_q1'] = df['stab_shap'].apply(lambda x: np.percentile(x, 25))  # Lower quartile (25th percentile)
    df['shap_q2'] = df['stab_shap'].apply(lambda x: np.percentile(x, 50))  # Median (50th percentile)
    df['shap_q3'] = df['stab_shap'].apply(lambda x: np.percentile(x, 75))  # Upper quartile (75th percentile)
    df['stability index'] = df['stab_shap'].apply(lambda x: np.mean(x))
    df['IQR'] = df['stab_shap'].apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))  # Interquartile range (Q3 - Q1)
    df['dataset'] = dataset_id
    df['hpo'] = np.where((df.n_estimators == 100) & (df.max_feats == df.max_feats.max()),
        'Benchmark',
        'Our model')

    # Select variables to keep
    df.rename(columns={'f1-score': 'f1_score'}, inplace=True)
    
    var = ['dataset', 'hpo', 'n_estimators', 'max_feats', 'n_feats', 
           'f1_score', 'precision', 'recall', 'stab_model', 'stab_shap', 
           'shap_q1', 'shap_q2', 'shap_q3', 'stability index', 'IQR', 'roc_auc']
    df = df[var]

    return df


def filter_and_select(df, fi_shap_all, boxplot=False, factor=0.99):
    """
    Filters and selects rows from a DataFrame based on specified conditions.
    
    Parameters:
    - df: A pandas DataFrame containing model evaluation metrics and parameters.
    
    Returns:
    - df_filtered: A filtered DataFrame based on the conditions.
    """
    # Define the variables of interest
    #var = ['dataset', 'hpo', 'n_estimators', 'n_feats', 'max_feats', 'roc_auc', 'precision', 'recall', 'f1_median', 'stability index', 'shap_stab']
    #df = df[var]

    #filter_shap = int(fi_shap_all[fi_shap_all.cum_value_percentage<99.9].n_feats.tail(1))

    df1_1 = df[df.hpo == 'Benchmark'].head(1)
    # Extract specific metrics from df1_1
    roc = df1_1.roc_auc.iloc[0] * factor
    precision = df1_1.precision.iloc[0] * factor
    recall = df1_1.recall.iloc[0] * factor
    f1_score = df1_1['f1_score'].iloc[0] * factor
    stability = df1_1['stability index'].iloc[0]
    IQR = df1_1['IQR'].iloc[0]
    n_features = df1_1.n_feats.iloc[0]

    # baseline = {'f1_score': f1_score, 'precision': precision, 'recall': recall, 'roc_auc': roc}
    baseline = {'precision': precision, 'roc_auc': roc}
    df1_2 = df[df.hpo != 'Benchmark']
    #criteria = np.where(df1_1.n_feats>=25, False, True)
    df1_2 = df1_2[df1_2['stability index'] >= stability]
    if n_features >= 10:
        df1_2 = df1_2[(df1_2.n_feats < n_features)]
    df1_2 = find_best_row_by_distance_and_stability(df1_2, 
                                                    baseline, 
                                                    stability_col='stability index',
                                                    IQR_col='IQR',
                                                    IQR_base=IQR,
                                                    distance_weight=1, 
                                                    stability_weight=0.3, 
                                                    threshold=np.inf)

    # Extract specific feature numbers
    dataframes = [df1_1, df1_2]
    df = pd.concat(dataframes)
    #df = df.drop_duplicates()

    return df


def find_best_row_by_distance_and_stability(df, baseline, stability_col='stability', IQR_col='IQR', IQR_base=np.inf, distance_weight=1, stability_weight=0.5, threshold=np.inf):
    """
    Finds the best matching row in df based on a combination of closeness to the baseline values, the stability score, IQR filtering,
    and optionally preferring rows with values higher than the baseline.
    
    Parameters:
    - df: DataFrame to search within.
    - baseline: Dictionary specifying baseline values for columns, excluding the stability and IQR columns.
    - stability_col: Name of the column containing stability scores.
    - IQR_col: Name of the column containing IQR values.
    - IQR_base: The maximum allowed IQR value for selection.
    - distance_weight: Weight of the distance score in the final score calculation.
    - stability_weight: Weight of the stability score in the final score calculation.
    - threshold: Maximum allowed final score to consider a row close enough.
    
    Returns:
    - A single-row DataFrame if a suitable row is found; otherwise, None.
    """
    if df.empty:
        return None

    # Check for missing columns
    missing_cols = [col for col in baseline.keys() if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in DataFrame: {missing_cols}")
    if stability_col not in df.columns or IQR_col not in df.columns:
        raise ValueError(f"Missing required columns in DataFrame")

    # First, try to filter rows with IQR <= IQR_base
    df_IQR_filtered = df[df[IQR_col] <= IQR_base]

    # If no rows fulfill the IQR requirement
    if df_IQR_filtered.empty:
        df_IQR_filtered = df
        baseline['IQR'] = IQR_base

    # First, try to filter rows with metrics >= baseline
    for metric, baseline_value in baseline.items():
        df_baseline_filtered = df_IQR_filtered[df_IQR_filtered[metric] >= baseline_value]
    
    # If no rows fulfill the metrics >= baseline requirement
    if df_baseline_filtered.empty:
        df_baseline_filtered = df_IQR_filtered
    
    # Compute the "distance" for each row in the filtered DataFrame
    distance = df_baseline_filtered.apply(lambda row: np.sqrt(sum((row[baseline.keys()] - pd.Series(baseline))**2)), axis=1)

    # Normalize distance and stability scores to [0, 1]
    distance_norm = (distance - distance.min()) / (distance.max() - distance.min())
    stability_norm = (df_baseline_filtered[stability_col] - df_baseline_filtered[stability_col].min()) / (df_baseline_filtered[stability_col].max() - df_baseline_filtered[stability_col].min())

    # Calculate combined score
    final_score = distance_weight * distance_norm - stability_weight * stability_norm

    # Find the row with the minimum final score within the threshold
    best_idx = final_score.idxmin()

    if final_score[best_idx] <= threshold:
        return df_baseline_filtered.loc[[best_idx]]
    else:
        return None


def process_graph_data(df):
    """
    Processes SHAP data files by reading them, filtering, and calculating additional metrics.
    
    Parameters:
    - dataset_id: The unique identifier for the dataset.
    - data_root: The root directory where the data files are located.
    
    Returns:
    - df: The processed pandas DataFrame containing selected variables and calculated metrics.
    """
    # Find the index of the maximum 'n_feat' in each 'n_estimators' and 'n_feats' group
    idx = df.groupby(['n_estimators', 'n_feats'])['max_feats'].idxmax()
    # Filter the DataFrame to keep only the rows with the maximum 'n_feat' per group
    df = df.loc[idx].reset_index(drop=True)

    return df


def append_dataframes(dataframes):
    """
    Appends a list of DataFrames into a single DataFrame.
    
    Parameters:
    - dataframes: List of pandas DataFrames with the same format.
    
    Returns:
    - A single DataFrame containing all rows from the input DataFrames.
    """
    # Concatenate the DataFrames row-wise
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

## Visualization of the results

In [6]:
import os
import pandas as pd

# Assuming 'data_root', 'process_data', 'append_dataframes', 'filter_and_select', 
# and 'process_graph_data' are defined elsewhere in your code.

# List of dataset identifiers
dataset_ids = ['arrhythmia', 'creditcard', 'musk', 'cardio', 'bank', 'mammography']

# Initialize lists to store dataframes
processed_dataframes = []
filtered_dataframes = []
graph_dataframes = []

# Loop through each dataset ID
for dataset_id in dataset_ids:
    # Construct file paths for SHAP values and feature importances
    path_shap = os.path.join(data_root, "outputs", f"{dataset_id}_shap.parquet")
    path_fi_shap = os.path.join(data_root, "outputs", f"{dataset_id}_fi_shap")
    
    # Read the parquet files
    df = pd.read_parquet(path_shap)
    fi_shap_all = pd.read_parquet(path_fi_shap)

    # Process the data
    processed_df = process_data(dataset_id, df)
    processed_dataframes.append(processed_df)
    
    # Filter and select data
    filtered_df = filter_and_select(processed_df, fi_shap_all, factor=1.0)
    filtered_dataframes.append(filtered_df)
    
    # Prepare graph data
    graph_df = process_graph_data(processed_df)
    graph_dataframes.append(graph_df)

# Append processed dataframes into a single dataframe
df = append_dataframes(processed_dataframes)

# Append filtered dataframes into a single dataframe for the group analysis
df_group = append_dataframes(filtered_dataframes)
var = ['dataset', 'hpo', 'n_estimators', 'max_feats', 'n_feats', 'roc_auc', 'precision', 'recall', 'f1_score',
       'stab_model', 'stability index', 'shap_q1', 'shap_q2', 'shap_q3', 'IQR', 'stab_shap']
df_group = df_group[var].round(4)

# Perform an inner join based on 'dataset', 'n_estimators', 'max_feats', and 'n_feats'
var = ['dataset', 'max_feats', 'n_feats']
df_benchmark = pd.merge(df, df_group[var][df_group.hpo=='Benchmark'], on=var, how='inner')
df_benchmark['hpo'] = 'Benchmark'
df_our_model = pd.merge(df, df_group[var][df_group.hpo=='Our model'], on=var, how='inner')
df_our_model['hpo'] = 'Our model'
df = df_benchmark.append(df_our_model, ignore_index=True)


# The graph dataframes are already prepared in the loop

FileNotFoundError: [Errno 2] No such file or directory: '/Users/eduardosepulveda/workspace_github/ad_shap_stability/test/data/outputs/creditcard_shap.parquet'

In [ ]:
df_group[['dataset', 'hpo', 'n_estimators', 'max_feats', 'n_feats', 'roc_auc', 'precision', 'recall', 'f1_score', 'stab_model', 'stability index', 'shap_q1', 'shap_q2', 'shap_q3', 'IQR']]

In [ ]:
# Example usage
image_1 =  '../boxplots.png'
boxplot_stability(df_group, size=1)
plt.savefig(image_1, bbox_inches='tight', pad_inches=0.1)  #Save the plot to a file

In [ ]:
test = df[(df.hpo=='Benchmark')] #((df.dataset=='arrhythmia') | (df.dataset=='creditcard')) & 
test[['dataset', 'hpo', 'max_feats', 'n_feats', 'n_estimators', 'stab_shap']]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import ast

# Datos de ejemplo en formato de texto para ser convertidos a un DataFrame
# Convertir el diccionario a un DataFrame
df = pd.DataFrame(test)

# Extraer los datos y las etiquetas
data = [df.loc[df['n_estimators'] == n, 'stab_shap'].values[0] for n in df['n_estimators']]
labels = [f'n_estimators={n}' for n in df['n_estimators']]

# Crear el gráfico de box-plots
plt.figure(figsize=(12, 8))
plt.boxplot(data, patch_artist=True, notch=True, vert=True)

# Añadir etiquetas a los ejes
plt.xticks(range(1, len(labels) + 1), labels, rotation=45)
plt.ylabel('stab_shap values')
plt.title('Box-Plots por n_estimators')

# Mostrar el gráfico
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Convertir el diccionario a un DataFrame
df = pd.DataFrame(test)

# Obtener la lista de datasets únicos
datasets = df['dataset'].unique()

# Crear la figura con subplots 2x3
fig, axs = plt.subplots(3, 2, figsize=(9, 8), sharex=True)

# Iterar sobre cada dataset y crear un subplot
for i, dataset in enumerate(datasets):
    row, col = divmod(i, 2)
    ax = axs[row, col]
    subset = df[df['dataset'] == dataset]
    data = [subset.loc[subset['n_estimators'] == n, 'stab_shap'].values[0] for n in subset['n_estimators']]
    labels = subset['n_estimators'].unique()
    
    ax.boxplot(data, patch_artist=False, notch=False, vert=True)
    ax.set_xticks(range(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    # ax.set_ylim(0, 1)  # Establecer los límites del eje Y de 0 a 1
    # ax.set_ylabel(f'stability measure')
    ax.set_title(f'{dataset}')

# Eliminar los subplots vacíos si hay menos de 6 datasets
if len(datasets) < 6:
    for j in range(len(datasets), 6):
        fig.delaxes(axs.flatten()[j])

# Añadir etiqueta al eje X
# Ajustar el layout para evitar solapamientos
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def boxplot_stability(df, ax, fontsize_title=8, fontsize_axes=6, stability_feat='stability index', boxplot_feat='stab_shap'):
    """
    Creates a boxplot where the X-axis is the number of estimators and the Y-axis is the stability index,
    with a boxplot of the specified feature for each value of n_estimators.

    Parameters:
    - df: DataFrame containing the columns 'n_estimators', 'stability index', and the feature to be plotted as boxplot.
    - ax: The matplotlib axes object where the plot will be drawn.
    - fontsize_title: Font size for the title.
    - fontsize_axes: Font size for the axes labels.
    - stability_feat: The feature to be plotted on the Y-axis.
    - boxplot_feat: The feature to be plotted as boxplot for each value of n_estimators.
    """
    # Ensure 'n_estimators' and 'stab_shap' are of correct data types
    df['n_estimators'] = df['n_estimators'].astype(str)
    df[boxplot_feat] = pd.to_numeric(df[boxplot_feat], errors='coerce')

    # Drop any rows with NaN values in 'stab_shap'
    df = df.dropna(subset=[boxplot_feat])

    # Create the boxplot
    sns.boxplot(x='n_estimators', y=boxplot_feat, data=df, ax=ax)

    # Set axis labels and title
    ax.set_xlabel('Number of Estimators', fontsize=fontsize_axes)
    ax.set_ylabel(stability_feat.capitalize(), fontsize=fontsize_axes)
    ax.set_title('Stability Index by Number of Estimators', fontsize=fontsize_title)

    # Set tick parameters
    ax.tick_params(axis='x', labelsize=fontsize_axes)
    ax.tick_params(axis='y', labelsize=fontsize_axes)

    # Show the plot
    plt.show()

In [ ]:
# Example usage
fig, ax = plt.subplots(figsize=(10, 6))
boxplot_stability(test, ax)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 25))  # Overall figure size

# Assuming col and row represent the number of columns and rows of subplots you want
col, row = 6, 1

# Loop through each dataset_id to create a subplot for each
for i, dataset_id in enumerate(dataset_ids, start=1):
    ax = plt.subplot(col, row, i)
    # Filter df_all for the current dataset_id and plot
    filtered_df = df[df.dataset == dataset_id]
    lineplot_stability(filtered_df, ax, primary_feat='f1_score', secondary_feat='stability index')

plt.tight_layout()  # Adjust layout to prevent overlap
plt.subplots_adjust(wspace=0.5, hspace=0.3)  # Adjust the spacing as needed

# Save the plot to a file
image_2 = '../comparison.png'
plt.savefig(image_2, bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
stop

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 25))  # Overall figure size

# Assuming col and row represent the number of columns and rows of subplots you want
col, row = 6, 1

# Loop through each dataset_id to create a subplot for each
for i, dataset_id in enumerate(dataset_ids, start=1):
    ax1 = plt.subplot(col, row, i)
    # Filter df_all for the current dataset_id and plot
    filtered_df = df_all[df_all.dataset == dataset_id]

    # Primary plot (e.g., precision)
    ax1.plot(filtered_df['x'], filtered_df['precision'], color='b', label='Precision')
    ax1.set_ylabel('Precision', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    # Create secondary y-axis
    ax2 = ax1.twinx()
    # Secondary plot (e.g., stability index)
    ax2.plot(filtered_df['x'], filtered_df['stability index'], color='r', label='Stability Index')
    ax2.set_ylabel('Stability Index', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    # Optionally set titles or legends
    ax1.set_title(f'Dataset ID: {dataset_id}')
    ax1.set_xlabel('X-axis label')  # Set the x-axis label if needed

plt.tight_layout()  # Adjust layout to prevent overlap
plt.subplots_adjust(wspace=0.5, hspace=1)  # Adjust the spacing as needed

# Save the plot to a file
image_2 = '../comparison.png'
plt.savefig(image_2, bbox_inches='tight', pad_inches=0.1)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10, 18))  # Overall figure size

col, row = 6, 1  # Number of columns and rows for subplots

for i, dataset_id in enumerate(dataset_ids, start=1):
    ax = plt.subplot(col, row, i)
    
    # Filter the DataFrame for the current dataset and model
    current_df = df[(df.dataset == dataset_id) & (df.hpo == 'Our model')]
    
    # Assuming 'stab_shap' is what you want to boxplot
    ax.boxplot(current_df['stab_shap'])
    
    # Set y-axis from 0 to 1
    # ax.set_ylim([0.3, 1])
    
    # Set x-axis labels to n_estimators values
    # This assumes you have the same number of n_estimators for each dataset_id. 
    # If not, you might need to adjust this part.
    ax.set_xticklabels(current_df['n_estimators'].unique(), rotation=0, ha="center")
    
    # Add a small title for each subplot
    ax.set_title(f"Dataset: {dataset_id}", fontsize=10)
    
    # Optional: Set labels for clarity
    ax.set_xlabel("n_estimators")
    ax.set_ylabel("Stability")

plt.tight_layout()  # Adjust layout to prevent overlap
plt.subplots_adjust(wspace=0.1, hspace=0.9)  # Adjust the spacing as needed

# Save the plot to a file
image_2 = '../comparison_boxplots.png'
plt.savefig(image_2, bbox_inches='tight', pad_inches=0.1)

plt.show()


In [ ]:
def process_data(dataset_id, df):
    """
    Processes SHAP data files by reading them, filtering, and calculating additional metrics including the interquartile range (IQR).
    
    Parameters:
    - dataset_id: The unique identifier for the dataset.
    - data_root: The root directory where the data files are located.
    
    Returns:
    - df: The processed pandas DataFrame containing selected variables and calculated metrics.
    """

    # Filter dataframe
    df = df[(df.n_estimators >= 25) & (df.n_estimators <= 300)]
    
    # Calculate percentiles, stability index, and IQR
    df['shap_q1'] = df['stab_shap'].apply(lambda x: np.percentile(x, 25))  # Lower quartile (25th percentile)
    df['shap_q2'] = df['stab_shap'].apply(lambda x: np.percentile(x, 50))  # Median (50th percentile)
    df['shap_q3'] = df['stab_shap'].apply(lambda x: np.percentile(x, 75))  # Upper quartile (75th percentile)
    df['stability index'] = df['stab_shap'].apply(lambda x: np.mean(x))
    df['IQR'] = df['stab_shap'].apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))  # Interquartile range (Q3 - Q1)
    df['dataset'] = dataset_id
    df['hpo'] = np.where((df.n_estimators == 100) & (df.max_feats == df.max_feats.max()),
        'Benchmark',
        'Our model')

    # Select variables to keep
    df.rename(columns={'f1-score': 'f1_score'}, inplace=True)
    
    var = ['dataset', 'hpo', 'n_estimators', 'max_feats', 'n_feats', 
           'f1_score', 'precision', 'recall', 'stab_model', 'stab_shap', 
           'shap_q1', 'shap_q2', 'shap_q3', 'stability index', 'IQR', 'roc_auc']
    df = df[var]

    return df

In [ ]:
dataset_id = 'allianz'
#df_names = [dataset_id1, dataset_id2, dataset_id3, dataset_id4, dataset_id5, dataset_id6]

path_shap = os.path.join(data_root, "outputs", f"{dataset_id}_shap.parquet")
path_fi_shap = os.path.join(data_root, "outputs", f"{dataset_id}_fi_shap.parquet")
# Read data
df = pd.read_parquet(path_shap)
fi_shap_all = pd.read_parquet(path_fi_shap)  # Note: fi_shap_all is read but not used in the provided code snippet

df_real = process_data(dataset_id, df)

factor = 1.0
#df_fil_real = filter_and_select(df_real, factor)

var = ['dataset', 'hpo', 'n_estimators', 'max_feats', 'n_feats', 'stability index', 'stab_shap', 'shap_q2', 'stab_model', 'IQR']
#df_group = round(df_fil_real[var],4)

df_graph_real = process_graph_data(df_real)

In [ ]:
fi_shap_all

In [ ]:
df_real

In [ ]:
group_1 = df_real[var][(df_real.n_estimators==100) & 
                       (df_real.n_feats==df_real.n_feats.max()) & 
                       (df_real.max_feats==df_real.max_feats.max())]

In [ ]:
group_1

In [ ]:
group_2 = df_real[var][(df_real.n_feats==34)].sort_values('stability index', ascending=False).head(1)
group_2

In [ ]:
df_group = append_dataframes([group_1, group_2])
df_group

In [ ]:
# Example usage
image_1 =  '../boxplots_real.png'
boxplot_stability(df_group, size=4)
#plt.savefig(image_1, bbox_inches='tight', pad_inches=0.1)  #Save the plot to a file

In [ ]:
stop

In [ ]:
plt.figure(figsize=(10, 20))  # Overall figure size

# First dataset
ax1 = plt.subplot(3, 2, 1, projection='3d')
plot_3d_surface(df1, 'n_estimators', 'n_feats', 'precision', ax1, fontsize_title=10, fontsize_axes=6, cmap='YlGnBu', x_step=25, y_step=50, opt_color='red', title=f'{dataset_id1}', alpha=0.85, edgecolor=None)
ax2 = plt.subplot(3, 2, 2, projection='3d')
plot_3d_surface(df1, 'n_estimators', 'n_feats', 'stability index', ax2, fontsize_title=10, fontsize_axes=6, cmap='YlGnBu', x_step=25, y_step=50, opt_color='red', title=f'{dataset_id1}', alpha=.9, edgecolor=None)

# Second dataset
ax1 = plt.subplot(3, 2, 3, projection='3d')
plot_3d_surface(df2, 'n_estimators', 'n_feats', 'precision', ax1, fontsize_title=10, fontsize_axes=6, cmap='YlGnBu', x_step=25, y_step=5, opt_color='red', title=f'{dataset_id2}', alpha=0.85, edgecolor=None)
ax2 = plt.subplot(3, 2, 4, projection='3d')
plot_3d_surface(df2, 'n_estimators', 'n_feats', 'stability index', ax2, fontsize_title=10, fontsize_axes=6, cmap='YlGnBu', x_step=25, y_step=5, opt_color='red', title=f'{dataset_id2}', alpha=.9, edgecolor=None)

# Third dataset
ax1 = plt.subplot(3, 2, 5, projection='3d')
plot_3d_surface(df3, 'n_estimators', 'n_feats', 'precision', ax1, fontsize_title=10, fontsize_axes=6, cmap='YlGnBu', x_step=25, y_step=50, opt_color='red', title=f'{dataset_id3}', alpha=0.85, edgecolor=None)
ax2 = plt.subplot(3, 2, 6, projection='3d')
plot_3d_surface(df3, 'n_estimators', 'n_feats', 'stability index', ax2, fontsize_title=10, fontsize_axes=6, cmap='YlGnBu', x_step=25, y_step=50, opt_color='red', title=f'{dataset_id3}', alpha=.9, edgecolor=None)

plt.tight_layout()  # Adjust layout


# Adjust the spacing between the plots
plt.subplots_adjust(wspace=0.1, hspace=0.1)  # Adjust the width space as needed for better visualization

#plt.savefig(image_1, bbox_inches='tight', pad_inches=0.3)  # Save the plot to a file
plt.show()